In [1]:
import pandas as pd
import numpy as np
import time
from transformers import AutoTokenizer, AutoModelForMaskedLM
from sentence_transformers import SentenceTransformer
from sentence_transformers import CrossEncoder
import torch

# Check Dataset

In [2]:
# Upload Dataset
dataset_dir = '../quran_100_ayat.csv'
df = pd.read_csv(dataset_dir)
df.head()

,id,suraId,verseID,ayahText,indoText,readText
0,0,1,1,بِسْمِ اللّٰهِ الرَّحْمٰنِ الرَّحِيْمِ,"Dengan nama Allah Yang Maha Pengasih, Maha Pen...",bismillāhir-raḥmānir-raḥīm
1,1,1,2,اَلْحَمْدُ لِلّٰهِ رَبِّ الْعٰلَمِيْنَۙ,"Segala puji bagi Allah, Tuhan seluruh alam,",al-ḥamdu lillāhi rabbil-'ālamīn
2,2,1,3,الرَّحْمٰنِ الرَّحِيْمِۙ,"Yang Maha Pengasih, Maha Penyayang,",ar-raḥmānir-raḥīm
3,3,1,4,مٰلِكِ يَوْمِ الدِّيْنِۗ,Pemilik hari pembalasan.,māliki yaumid-dīn
4,4,1,5,اِيَّاكَ نَعْبُدُ وَاِيَّاكَ نَسْتَعِيْنُۗ,Hanya kepada Engkaulah kami menyembah dan hany...,iyyāka na'budu wa iyyāka nasta'īn


In [3]:
# Check Word Count
df['word_count'] = df['indoText'].apply(lambda x: len(str(x).split()))
print(df['word_count'].describe())

count    100.000000
mean      28.660000
std       17.114127
min        3.000000
25%       18.000000
50%       25.500000
75%       37.250000
max      107.000000
Name: word_count, dtype: float64


# Check Model 

In [4]:
output_dir ='../mono to multi lingual/output/make-multilingual-en-id-2025-03-19_13-10-48/final'
model = SentenceTransformer(output_dir, 
                            device='cuda', 
                            model_kwargs={'torch_dtype':'bfloat16'} #speed up inference on GPUs at a minimal loss of model accuracy
                            )

In [5]:
# The sentences to encode
sentences = [
    "The weather is lovely today.",
    "It's so sunny outside!",
    "He drove to the stadium.",
]

embeddings = model.encode(sentences)
print(embeddings.shape)

(3, 768)


In [6]:
embeddings

array([[-0.10595703,  0.03088379, -0.04931641, ...,  0.17578125,
         0.01586914,  0.1875    ],
       [-0.234375  , -0.00518799, -0.10791016, ...,  0.2890625 ,
         0.02148438,  0.25      ],
       [-0.28125   , -0.28125   , -0.20800781, ...,  0.18554688,
         0.07128906,  0.0378418 ]], shape=(3, 768), dtype=float32)

# Check Similarity

In [7]:
query = 'apa itu jalan yang lurus?'
queries = [
    'apa itu jalan yang lurus?',
    'siapa yang berbohong?'
]

In [ ]:
embeddings_query = model.encode(queries, output_value='token_embeddings')
embeddings_query

In [12]:
# encode dataset
embeddings_dataset = model.encode(df['indoText'])
embeddings_dataset.shape

(100, 768)

In [ ]:
# Compute similarities
similarities = model.similarity(embeddings_query, embeddings_dataset)

# Output the pairs with their score
print(query)
for idx_i, sentence in enumerate(df['indoText']):
    print(f"{similarities[0][idx_i]:.4f} : {sentence: <30}")

apa itu jalan yang lurus?
0.4016 : Dengan nama Allah Yang Maha Pengasih, Maha Penyayang.
0.4514 : Segala puji bagi Allah, Tuhan seluruh alam,
0.6738 : Yang Maha Pengasih, Maha Penyayang,
0.8203 : Pemilik hari pembalasan.      
0.6815 : Hanya kepada Engkaulah kami menyembah dan hanya kepada Engkaulah kami mohon pertolongan.
0.8270 : Tunjukilah kami jalan yang lurus,
0.6991 : (yaitu) jalan orang-orang yang telah Engkau beri nikmat kepadanya; bukan (jalan) mereka yang dimurkai, dan bukan (pula jalan) mereka yang sesat.
0.7124 : Alif Lam Mim.                 
0.6570 : Kitab (Al-Qur'an) ini tidak ada keraguan padanya; petunjuk bagi mereka yang bertakwa,
0.6242 : (yaitu) mereka yang beriman kepada yang gaib, melaksanakan salat, dan menginfakkan sebagian rezeki yang Kami berikan kepada mereka,
0.5510 : dan mereka yang beriman kepada (Al-Qur'an) yang diturunkan kepadamu (Muhammad) dan (kitab-kitab) yang telah diturunkan sebelum engkau, dan mereka yakin akan adanya akhirat.
0.6202 : Merekalah y

## Sliding Window

In [31]:
indoTexts = df['indoText'].tolist()
indoTexts

['Dengan nama Allah Yang Maha Pengasih, Maha Penyayang.',
 'Segala puji bagi Allah, Tuhan seluruh alam,',
 'Yang Maha Pengasih, Maha Penyayang,',
 'Pemilik hari pembalasan.',
 'Hanya kepada Engkaulah kami menyembah dan hanya kepada Engkaulah kami mohon pertolongan.',
 'Tunjukilah kami jalan yang lurus,',
 '(yaitu) jalan orang-orang yang telah Engkau beri nikmat kepadanya; bukan (jalan) mereka yang dimurkai, dan bukan (pula jalan) mereka yang sesat.',
 'Alif Lam Mim.',
 "Kitab (Al-Qur'an) ini tidak ada keraguan padanya; petunjuk bagi mereka yang bertakwa,",
 '(yaitu) mereka yang beriman kepada yang gaib, melaksanakan salat, dan menginfakkan sebagian rezeki yang Kami berikan kepada mereka,',
 "dan mereka yang beriman kepada (Al-Qur'an) yang diturunkan kepadamu (Muhammad) dan (kitab-kitab) yang telah diturunkan sebelum engkau, dan mereka yakin akan adanya akhirat.",
 'Merekalah yang mendapat petunjuk dari Tuhannya, dan mereka itulah orang-orang yang beruntung.',
 'Sesungguhnya orang-orang

In [43]:
window_size = 3
passages = []
for start_idx in range(0, len(indoTexts)):
        end_idx = min(start_idx + window_size, len(indoTexts))
        passages.append(" ".join(indoTexts[start_idx:end_idx]))

passages

['Dengan nama Allah Yang Maha Pengasih, Maha Penyayang. Segala puji bagi Allah, Tuhan seluruh alam, Yang Maha Pengasih, Maha Penyayang,',
 'Segala puji bagi Allah, Tuhan seluruh alam, Yang Maha Pengasih, Maha Penyayang, Pemilik hari pembalasan.',
 'Yang Maha Pengasih, Maha Penyayang, Pemilik hari pembalasan. Hanya kepada Engkaulah kami menyembah dan hanya kepada Engkaulah kami mohon pertolongan.',
 'Pemilik hari pembalasan. Hanya kepada Engkaulah kami menyembah dan hanya kepada Engkaulah kami mohon pertolongan. Tunjukilah kami jalan yang lurus,',
 'Hanya kepada Engkaulah kami menyembah dan hanya kepada Engkaulah kami mohon pertolongan. Tunjukilah kami jalan yang lurus, (yaitu) jalan orang-orang yang telah Engkau beri nikmat kepadanya; bukan (jalan) mereka yang dimurkai, dan bukan (pula jalan) mereka yang sesat.',
 'Tunjukilah kami jalan yang lurus, (yaitu) jalan orang-orang yang telah Engkau beri nikmat kepadanya; bukan (jalan) mereka yang dimurkai, dan bukan (pula jalan) mereka yang s

### Bi Encoder

In [56]:
passage_embeddings = model.encode(passages)
passage_embeddings

array([[-0.58545464,  0.34384415, -0.78585005, ..., -0.30037302,
         0.06373288,  0.57276285],
       [-0.5934459 ,  0.3516204 , -0.7357356 , ..., -0.2063958 ,
         0.10635417,  0.603396  ],
       [-0.329024  ,  0.1392722 , -0.18247563, ...,  0.29004464,
         0.18761168,  0.7073407 ],
       ...,
       [-0.39760172,  0.134579  , -0.4420009 , ..., -0.02436221,
         0.14721079,  0.6934576 ],
       [-0.21743368, -0.1376738 , -0.11424442, ...,  0.10803927,
         0.05506072,  0.31996584],
       [-0.21449426, -0.08332802, -0.03698666, ...,  0.20873895,
         0.05871074,  0.35652125]], shape=(100, 768), dtype=float32)

In [59]:
query_embeddings = model.encode(query)

In [65]:
sorted_indices = torch.argsort(similarities, descending=True)
sorted_indices

tensor([[ 5,  4, 30, 41, 98, 99, 64, 74, 63,  3, 57, 36, 62, 40, 37, 35,  2, 56,
          6, 38, 39, 59,  7, 76, 44, 79, 92, 58,  8, 45, 66, 75, 67,  9, 52, 54,
         53, 55, 65, 46, 31, 42, 77, 78, 90, 73, 17, 69, 71, 34, 23, 91, 24, 72,
         25, 80, 10, 47, 18, 26, 61, 27, 68, 70, 43, 22, 60, 48, 28, 29, 87, 33,
         51, 21, 97, 93, 88,  1, 50, 89, 20, 32, 86, 16, 19, 96, 82, 11, 83, 49,
         13, 84, 95, 81,  0, 85, 12, 94, 15, 14]])

In [76]:
similarities = model.similarity(query_embeddings, passage_embeddings)


print(query)
for idx_i, sentence in enumerate(passages):
    print(f"{similarities[0][idx_i]:.4f} : {sentence: <30}")

apa itu jalan yang lurus?
0.3332 : Dengan nama Allah Yang Maha Pengasih, Maha Penyayang. Segala puji bagi Allah, Tuhan seluruh alam, Yang Maha Pengasih, Maha Penyayang,
0.3859 : Segala puji bagi Allah, Tuhan seluruh alam, Yang Maha Pengasih, Maha Penyayang, Pemilik hari pembalasan.
0.6171 : Yang Maha Pengasih, Maha Penyayang, Pemilik hari pembalasan. Hanya kepada Engkaulah kami menyembah dan hanya kepada Engkaulah kami mohon pertolongan.
0.6459 : Pemilik hari pembalasan. Hanya kepada Engkaulah kami menyembah dan hanya kepada Engkaulah kami mohon pertolongan. Tunjukilah kami jalan yang lurus,
0.7149 : Hanya kepada Engkaulah kami menyembah dan hanya kepada Engkaulah kami mohon pertolongan. Tunjukilah kami jalan yang lurus, (yaitu) jalan orang-orang yang telah Engkau beri nikmat kepadanya; bukan (jalan) mereka yang dimurkai, dan bukan (pula jalan) mereka yang sesat.
0.7295 : Tunjukilah kami jalan yang lurus, (yaitu) jalan orang-orang yang telah Engkau beri nikmat kepadanya; bukan (jalan) 

### CrossEncoder

In [ ]:
modelCrossEncoder = CrossEncoder('cross-encoder/ms-marco-TinyBERT-L2', device='cuda')

for query in queries :
    start_time = time.time()

    model_inputs = [[query, passage] for passage in passages]
    scores = modelCrossEncoder.predict(model_inputs)

    # Sort the scores in decreasing order
    results = [{"input": inp, "score": score} for inp, score in zip(model_inputs, scores)]
    results = sorted(results, key=lambda x: x["score"], reverse=True)

    print("Query:", query)
    print(f"Search took {time.time() - start_time:.2f} seconds")
    for hit in results[0:5]:
        print("Score: {:.2f}".format(hit["score"]), "\t", hit["input"][1])

    print("==========")

Query: apa itu jalan yang lurus?
Search took 0.10 seconds
Score: 0.90 	 Tunjukilah kami jalan yang lurus, (yaitu) jalan orang-orang yang telah Engkau beri nikmat kepadanya; bukan (jalan) mereka yang dimurkai, dan bukan (pula jalan) mereka yang sesat. Alif Lam Mim.
Score: 0.87 	 (yaitu) orang-orang yang melanggar perjanjian Allah setelah (perjanjian) itu diteguhkan, dan memutuskan apa yang diperintahkan Allah untuk disambungkan dan berbuat kerusakan di bumi. Mereka itulah orang-orang yang rugi. Bagaimana kamu ingkar kepada Allah, padahal kamu (tadinya) mati, lalu Dia menghidupkan kamu, kemudian Dia mematikan kamu lalu Dia menghidupkan kamu kembali. Kemudian kepada-Nyalah kamu dikembalikan. Dialah (Allah) yang menciptakan segala apa yang ada di bumi untukmu kemudian Dia menuju ke langit, lalu Dia menyempurnakannya menjadi tujuh langit. Dan Dia Maha Mengetahui segala sesuatu.
Score: 0.78 	 Dan berimanlah kamu kepada apa (Al-Qur'an) yang telah Aku turunkan yang membenarkan apa (Taurat) yan

# Check Multilingual Model Effect

In [2]:
dataset_dir = '../quran_id_full.csv'
df = pd.read_csv(dataset_dir)

In [7]:
query = 'apa itu jalan yang lurus?'

## Bert Base

## XLM RoBERTa
multilingual model default

In [13]:
tokenizer = AutoTokenizer.from_pretrained('xlm-roberta-base')
model_xlm = AutoModelForMaskedLM.from_pretrained("xlm-roberta-base")

Some weights of the model checkpoint at xlm-roberta-base were not used when initializing XLMRobertaForMaskedLM: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMRobertaForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [14]:
embeddings_dataset_xlm = model_xlm.encode(df['indoText'])
embeddings_dataset_xlm.shape

AttributeError: 'XLMRobertaForMaskedLM' object has no attribute 'encode'

## Multilingual XLM RoBERTa Trained

In [68]:
output_model_multilang_dir ='../mono to multi lingual/output/make-multilingual-en-id-2025-03-19_13-10-48/final'
model_multilang = SentenceTransformer(output_model_multilang_dir, 
                            device='cuda', 
                            model_kwargs={'torch_dtype':'bfloat16'} #speed up inference on GPUs at a minimal loss of model accuracy
                            )

In [69]:
# encode dataset
embeddings_dataset_multilang = model_multilang.encode(df['indoText'])
embeddings_dataset_multilang.shape

(6236, 768)

In [70]:
embeddding_query_multilang = model_multilang.encode(query)

In [112]:
similarities_multilang = model_multilang.similarity(embeddding_query_multilang, 
                                                    embeddings_dataset_multilang)
similarities_multilang

tensor([[0.3993, 0.4526, 0.6704,  ..., 0.7485, 0.7888, 0.9165]])

In [84]:
sorted_values_multilang, sorted_indices_multilang = torch.sort(similarities_multilang, descending=True)
sorted_indices_multilang = sorted_indices_multilang.tolist()[0]
sorted_indices_multilang

[5909,
 5941,
 3113,
 4735,
 5884,
 5829,
 4681,
 6099,
 6100,
 4919,
 4740,
 5886,
 5643,
 5411,
 5344,
 5526,
 4737,
 5630,
 6031,
 5810,
 5009,
 5623,
 5653,
 5771,
 5852,
 5010,
 6127,
 5984,
 4790,
 5715,
 4798,
 4898,
 3851,
 5976,
 3065,
 1839,
 5986,
 3679,
 5533,
 5788,
 6176,
 5634,
 5913,
 6235,
 4007,
 5716,
 6098,
 5519,
 6047,
 4335,
 5663,
 5801,
 5323,
 6157,
 4962,
 5646,
 5678,
 5520,
 3852,
 6167,
 5978,
 5358,
 5033,
 5005,
 4858,
 3126,
 5985,
 5712,
 5559,
 5324,
 5959,
 5996,
 5345,
 3829,
 5015,
 6158,
 6162,
 3680,
 5072,
 4303,
 5016,
 4680,
 5805,
 5503,
 5741,
 5018,
 1820,
 2147,
 6155,
 5030,
 3705,
 2466,
 6048,
 5893,
 6140,
 4678,
 5558,
 5511,
 5800,
 5502,
 5900,
 6227,
 5812,
 6132,
 3427,
 6164,
 5071,
 5541,
 6173,
 5696,
 4466,
 5856,
 5867,
 6202,
 3708,
 6192,
 2969,
 4675,
 6218,
 4358,
 6187,
 4336,
 5793,
 5690,
 5021,
 5811,
 6095,
 2903,
 3836,
 4640,
 3833,
 5773,
 5692,
 5545,
 2353,
 4954,
 6063,
 3831,
 5677,
 5486,
 6148,
 5830,
 5930,

In [85]:

for idx, value in enumerate(sorted_indices_multilang):
    print(f'{idx} {sorted_values_multilang[0][idx]:.4f} {df['indoText'][value]: <30}')

0 0.9665 Demi langit yang mempunyai gugusan bintang,
1 0.9652 Demi langit yang mengandung hujan,
2 0.9640 Dan timbanglah dengan timbangan yang benar.
3 0.9625 Demi gunung (Sinai),          
4 0.9595 Apabila langit terbelah,      
5 0.9595 Apabila langit terbelah,      
6 0.9592 Demi langit yang mempunyai jalan-jalan,
7 0.9581 demi gunung Sinai,            
8 0.9581 dan demi negeri (Mekah) yang aman ini.
9 0.9496 Dia membiarkan dua laut mengalir yang (kemudian) keduanya bertemu,
10 0.9470 demi lautan yang penuh gelombang,
11 0.9469 dan apabila bumi diratakan,   
12 0.9456 sampai waktu yang ditentukan, 
13 0.9434 dari kanan dan dari kiri dengan berkelompok-kelompok?
14 0.9433 dalam surga yang tinggi,      
15 0.9430 Tidak! Demi bulan,            
16 0.9420 pada lembaran yang terbuka,   
17 0.9413 dan apabila langit terbelah,  
18 0.9412 dan lidah dan sepasang bibir? 
19 0.9373 dan apabila langit dilenyapkan,
20 0.9354 dan air yang mengalir terus-menerus,
21 0.9346 dan (malaikat-malaikat)

# Check Sliding Window (SW) Effect

In [ ]:
window_size = 3
passages = []
for start_idx in range(0, len(df['indoText'])):
        end_idx = min(start_idx + window_size, len(df['indoText']))
        passages.append(" ".join(df['indoText'][start_idx:end_idx]))


In [108]:
df['text sliding window'] = df.apply(lambda x: passages[x.name], axis=1)

In [109]:
# encode dataset
embeddings_dataset_sw = model_multilang.encode(df['text sliding window'] )
embeddings_dataset_sw.shape

(6236, 768)

In [110]:
embeddding_query_sw = model_multilang.encode(query)

In [111]:
similarities_sw = model_multilang.similarity(embeddings_dataset_sw, 
                                             embeddding_query_sw)
similarities_sw

tensor([[0.3318],
        [0.3830],
        [0.6136],
        ...,
        [0.7344],
        [0.7748],
        [0.9165]])

In [113]:
sorted_values_sw, sorted_indices_sw = torch.sort(similarities_sw, descending=True)

In [65]:

for idx, value in enumerate(sorted_indices_sw):
    print(f'{idx} {sorted_values_sw[0][idx]:.4f} {passages[value]: <30}')

0 0.3318 Dengan nama Allah Yang Maha Pengasih, Maha Penyayang. Segala puji bagi Allah, Tuhan seluruh alam, Yang Maha Pengasih, Maha Penyayang,


IndexError: index 1 is out of bounds for dimension 0 with size 1